In [1]:
import numpy as np
import matplotlib.pylab as plt
from pylab import *
import sys
import pandas as pd

In [2]:
##########################
# define a few constants #		##fft python random phase
##########################

e0 = np.loadtxt('837nm.dat')		#load the reference pulse, pump @836 nm
e0 = e0.T
e1 = np.sqrt(e0[1])
e1_ph = e0[2]
e = np.array(e1)*np.exp(-1j*(np.array(e1_ph)))		#reforming E-field

original_trace = np.loadtxt('trace/trace1.frg')		#load FROG trace from experiment

N = len(e0[0])		#timesteps
N_max = np.max(e0[0])
N_min = np.min(e0[0])

t = e0[0]-N_min
T0 = N_max-N_min	#total time duration
dt = T0/(N-1)
df = 1.0/T0		#frequency steps
TD = N			#time delay for FROG-trace generation
freq = np.arange(0,1/dt+df,df)*1000

In [3]:
original_trace.shape

(128, 128)

In [4]:
## for generation of Gaussian pulses
def gaussian_pulse(T,f,A,phase,tau):
	w0 = f*2*np.pi		#central angular frequency of pulse
	temp = -1j*w0*(t-tau) + 1j*phase
	E = A*np.sqrt(np.exp(-4.0*np.log(2.0)/np.log(np.e)*((t-tau)/T)**2))*np.exp(-temp)
	return E

In [5]:
## calculate cross correlation of two pulses
def xcorr(e0,e,td):		#shift one pulse by time delay of 'td'
	E_delay = e*1.0
	if td<=0:
		E_delay[0:N+td] = e0[-td:]
		E_delay[N+td:] = [0]*(-td)
	else:
		E_delay[0:td] = [0]*(td)
		E_delay[td:] = e0[0:N-td]	
	return  e*E_delay

In [6]:
## calculate the differert of trace A and trace B
def array_diff(A,B):
	A = A/np.max(A)
	B = B/np.max(B)
	return np.sum(np.square(A-B))/np.sum(np.square(A))

In [7]:
def FROG_generator(e,E,TD):	
	FROG = np.zeros((TD,N),dtype=float)
	for td in range(-TD//2,TD//2):
		E_corre = xcorr(e,E,td)
		temp0 = np.fft.fftshift(np.fft.fft(E_corre))
		FROG[(td+TD//2)-1] = abs(temp0)**2	
	return FROG.T

In [8]:
## plot out FROG trace
def FROG_trace_plot(FROG):
	fig,ax = plt.subplots()
	plt.imshow(FROG)
	plt.title("FROG trace")
	plt.grid(True)
	plt.show()

In [22]:
## calculate FROG traces with different time delays
def FROGs(T1,T2,A1,A2,a2,a3,f0,f1,tau1):
    tau2 = tau1-700
    phase1 = a2*(t-tau1)**2+a3*(t-tau1)**3
    phase2 = a2*(t-tau2)**2+a3*(t-tau2)**3 
    E1 = gaussian_pulse(T1,f0,A1,phase1,tau1)
    E2 = gaussian_pulse(T2,f1,A2,phase2,tau2)
    E = E1+E2
    FROG = FROG_generator(e,E,TD)
    delta = array_diff(FROG, original_trace)	
    return FROG, delta

In [23]:
######################
# initial parameters #
######################

T1 = np.arange(295,695)                  #Gaussian pulse duration in fs, FW
T2 = np.arange(300,1000)
#T2 = 287.7733748374851
A1 = np.arange(0.6,1.5,0.05)
#A1 = 0.7524039393439459                #Amplitude of Raman pulse
A2 = np.arange(0.6,1.5,0.05)
#A2 = 1.369630361735096

a2 = np.arange(-4e-6, 1e-6, 1e-8)
a2 = np.delete(a2, np.where(np.abs(a2) < 1e-10))#second order phase factor
a3 = np.arange(-1.45e-9, 1.5e-9, 1e-11)		#third order phase factor
a3 = np.delete(a3, np.where(np.abs(a3) < 1e-14))

F0 = np.arange(-0.005, 0.005, 0.0005)
F0 = np.delete(F0, np.where(np.abs(F0) < 1e-10))    #Raman center-wavelength
#f0 = 0.0014144712047722094
F1 = np.arange(-0.0005, 0.0005, 0.00005)
F1 = np.delete(F1, np.where(np.abs(F1) < 1e-10))    #Red-shifted center-wavelength
#f1 = 0.0002521943119991874
			
i0 = 333.0		#unit: fs
Tau1 = np.arange(-1000, 5000, 50)	#Raman pulse peak time
#tau1 = 3325.2480418676446
#tau2 = tau1-700		#Red-shifted pulse peak time

In [24]:
a3

array([-1.45e-09, -1.44e-09, -1.43e-09, -1.42e-09, -1.41e-09, -1.40e-09,
       -1.39e-09, -1.38e-09, -1.37e-09, -1.36e-09, -1.35e-09, -1.34e-09,
       -1.33e-09, -1.32e-09, -1.31e-09, -1.30e-09, -1.29e-09, -1.28e-09,
       -1.27e-09, -1.26e-09, -1.25e-09, -1.24e-09, -1.23e-09, -1.22e-09,
       -1.21e-09, -1.20e-09, -1.19e-09, -1.18e-09, -1.17e-09, -1.16e-09,
       -1.15e-09, -1.14e-09, -1.13e-09, -1.12e-09, -1.11e-09, -1.10e-09,
       -1.09e-09, -1.08e-09, -1.07e-09, -1.06e-09, -1.05e-09, -1.04e-09,
       -1.03e-09, -1.02e-09, -1.01e-09, -1.00e-09, -9.90e-10, -9.80e-10,
       -9.70e-10, -9.60e-10, -9.50e-10, -9.40e-10, -9.30e-10, -9.20e-10,
       -9.10e-10, -9.00e-10, -8.90e-10, -8.80e-10, -8.70e-10, -8.60e-10,
       -8.50e-10, -8.40e-10, -8.30e-10, -8.20e-10, -8.10e-10, -8.00e-10,
       -7.90e-10, -7.80e-10, -7.70e-10, -7.60e-10, -7.50e-10, -7.40e-10,
       -7.30e-10, -7.20e-10, -7.10e-10, -7.00e-10, -6.90e-10, -6.80e-10,
       -6.70e-10, -6.60e-10, -6.50e-10, -6.40e-10, 

In [27]:
values = pd.DataFrame(columns = ['T1', 'T2', 'A1', 'A2', 'a2', 'a3', 'F0', 'F1', 'Tau1'])
values

,T1,T2,A1,A2,a2,a3,F0,F1,Tau1


In [28]:
###########################
# FROG trace main program #
###########################
FROGS = []
values = []
for i in range(5000):
    t1 = np.random.choice(T1)
    t2 = np.random.choice(T2)
    ai1 = np.random.choice(A1)
    ai2 = np.random.choice(A2)
    f0 = np.random.choice(F0)
    f1 = np.random.choice(F1)
    aa2 = np.random.choice(a2)
    aa3 = np.random.choice(a3)
    tau1 = np.random.choice(Tau1)
    (FROG, delta) = FROGs(t1,t2,ai1,ai2,aa2,aa3,f0,f1,tau1)
    print(delta)
    values.append([t1, t2, ai1, ai2, aa2, aa3, f0, f1, tau1])
    FROGS.append(FROG)

0.9352855253424703
2.552552850896584
2.9035980954995626
2.868694265940375
2.383086340843026
1.6800230545879258
2.22874291826379
1.4901593806306779
1.1368534128847565
1.9181172943752938
2.5285044469704023
1.9796383570801077
2.4845999733345554
2.307674892821762
1.462891795460379
1.9048455783746412
2.29605602490275
1.5645508431018946
0.7706534186558788
1.1563756126308073
1.2423581299871986
2.7461570072058703
2.479114888179762
2.0333339367004255
1.7278582255629282
2.1524026639972367
2.622594718209126
1.1280841707107825
1.8493051311320365
2.441973380829489
1.8303527054965922
1.6905582785344004
1.8747413665389088
0.6160309607977543
1.5745624847681854
1.5421364239149646
1.5846861550079216
1.9156278797522057
2.0484473416541373
1.6862766339544832
2.651241577159116
0.9090164618017731
1.8874348375038301
2.168500988349934
1.9496616242757463
2.1175894762428364
2.2552845214647594
2.567741708401534
1.5139077303076751
0.6550771128651913
1.1430127990938177
1.9148335673889054
1.5662506635137579
1.617006

0.655536162736889
2.1514386653561894
2.5661077987137677
0.4451832128889129
1.4235605212488902
1.2090899215611584
1.0744271975516533
1.9717181047121006
2.0727867625320826
1.9691947093602031
2.9220658485603432
1.452408731349899
1.2766811600565426
1.569158820085732
1.8082084092838333
1.6049184950345996
2.125853416802324
1.0964507437517372
2.1713304869559225
1.6958450946039758
2.164116645572513
1.1785250426283504
1.7935431506696862
1.7106053705260127
1.5220950373909066
0.8938302505439806
1.9707519383038037
2.2204158837357473
2.251082704785396
2.062103558323059
1.282035133229669
2.455792506109896
1.0718184548764387
1.4640942769004814
1.6051152834235018
2.5773852627925913
1.6910654234341376
1.0533480891841882
2.623910345803066
0.81722031936525
2.238911469792313
1.4750854395303623
2.2197974333306463
2.3033117393630533
2.784935003309038
2.237265465692265
3.145579908282339
2.211251885137631
2.5328164633534116
1.9022572077737616
1.8689671475459009
2.7326635496891236
1.7435057213170713
1.90096392

2.3262456660633597
2.4133368692487323
1.1450263897841704
2.6246023154022367
2.8365736381195026
2.0429044252310398
0.7068376546459131
2.0148829098091707
1.9185016271471256
1.5222141495552117
1.4157863089024798
0.7462110550179891
2.2081386194260264
2.785368184610386
2.29931960764202
2.0309245214679503
1.7971978639330322
2.022466482150049
1.788917604067563
2.304966659351504
2.0762781085957007
2.271558641241235
2.2580062981567006
2.4484842635748256
1.7661202110571748
2.2358733170176865
2.0027270229626963
2.2689284724913223
1.2644698291572976
2.7432638752471594
0.7621283256682804
2.476468701754454
2.6613966348364455
1.6448684728516152
0.8027876162320775
2.0603407939931917
2.5781874136573015
2.567214837321416
2.6417452765120135
0.8937661768984151
2.0334782417585666
2.4387684912557854
2.3506149007086163
2.5551617724837454
0.9708981621651381
1.9216909732260472
2.4009218487378186
2.0138147863974583
1.7094098479532474
1.8213737314406304
2.4578519889321124
1.507241675531433
1.0337317612969366
1.2

2.04576339638832
2.530482666406234
2.1016933577838675
1.8289226451934182
2.7865425111481756
2.185266194659422
2.317861193185188
1.1412407430728524
2.1311209713557786
2.534869604234964
2.4038741461931687
1.531506468143325
1.7228453506518822
2.3591141968665146
1.987439427355223
2.6034238475909306
2.4140406922522377
1.5547115756879384
1.835876751524838
1.8871888695306815
1.5949171167808236
1.526318784118818
0.5500189134617578
2.1335343666369484
2.1382112243697917
2.407449127748292
2.7058231427111528
2.121075198539033
2.628666649299901
2.7900000416743582
2.1306283515579767
2.2096753152825257
1.7259707303784524
1.8827923656886585
2.7108771118293102
1.9053963650191281
1.8165093722007546
2.4295278573319075
2.667513315292255
1.759837565890789
1.8847163047191957
1.827913777343359
2.9098980852430985
1.02672332277798
1.9198998150134856
2.300099568718016
2.0518918629387946
1.7977122704153792
1.481428184923926
1.9320085386681678
2.656206424116001
1.9434932793672908
1.7104554238271081
2.022877503363

1.657066638591713
2.436252675493097
1.8399511684383878
1.9120666473479426
1.7566728085847545
1.8697882394850172
2.312951642713523
1.724152759839813
2.350846112962438
1.7589959932126589
0.8758452851680343
1.422740562698446
1.003251321407319
2.284315716581316
2.135525798397432
1.6117725613159513
1.353054770313697
1.5077209487258216
1.9949360332639547
2.582988857447698
1.3689273125911898
0.8139825181644987
1.5230804506906528
2.187614596288248
0.7609911988767493
2.330953340926128
1.5661479674917107
2.084441654117263
2.047266966919498
2.3847080303217254
2.1000471638762996
1.6922830233336337
2.439072846629912
2.1545665811760553
1.7371235257603117
2.0254885734996844
0.9236448092861325
2.136023611198002
2.1610203654501814
1.7843025710827365
2.5233696824651752
0.9021115188303748
2.107578891611427
2.3507829747980167
2.019821331693077
1.8936184886038367
0.8415998107020067
2.5974625190211613
1.4029071344317519
0.7000146068462897
0.5304886362378038
1.9684969071758023
1.1493695533617962
1.4320989565

...

2.2257678446835083
1.6622309623635034
1.9431659132505068
2.3129318247202226
1.9392652670047699
1.7734686541924876
0.7449444820624237
1.3928427825085097
2.764680888449846
2.0566907722769106
1.9244304402402386
1.4837116769216319
1.9050545129843042
0.952479077193834
2.2373456284400226
1.9743558463836912
2.6508121760132606
1.5980136278515134
1.8737460348240251
2.437356558087931
2.2618681658238535
2.3109126699845577
1.7502985295279292
1.9647516548356903
1.4502543590387098
2.1270102374548534
2.083606205302243
0.4685970238304867
2.4722296042991534
2.4346440314112843
1.7012529123386655
2.519825726672525
2.5575953936055504
2.6506073259340694
1.4103617885612816
2.321287511972308
1.8690844412609833
1.3502755923735605
1.2635165480028343
2.2411628855743517
2.1825021651228886
2.014048444119213
2.4260596555232126
1.8191496793636701
1.5614232926287939
1.3823029711842652
1.7014672165410163
0.7983083687820226
2.513567084180513
0.8753566070554314
2.0093882514890966
1.291909371950993
1.5057612660671802
3.

2.401543821571919
2.003254041232503
1.4505756176895637
1.1906771080598504
2.2901938024375412
2.500209195145641
1.4658372696234605
2.491906680588038
1.1000059176546522
1.8659404919393838
0.8090045563041384
1.0706300810298588
2.3071853099133994
2.270162691734502
2.2266031423187416
2.2423876642314116
1.772352593337466
0.6488360546109316
2.168511784006872
0.8314446015744877
2.2787617323946896
2.6394944407738556
2.2261614702792576
2.1204163918175554
1.8055782860281908
0.9600294503499961
0.9140026792442503
1.7659323077493172
2.154204689336743
2.4281538303936
0.6328494367524393
0.9316112633945327
2.6495863159359296
2.4620987412478925
1.416475642268021
3.09539215298237
1.2329432416975352
2.579241288121767
1.2210540228667162
2.608319185460316
1.8200560692966758
1.5555827762678678
1.3861880616098305
1.461576325222954
1.5931661821419347
1.0880598790172575
2.1535006770030036
0.603339001016569
2.6025304657578903
2.146476619871811
2.0966826503322045
1.9359859920715587
2.3367585429497786
1.7907455681

1.240257695432479
1.7944109676522098
0.7732000343231045
0.6359082355946403
2.3134270404597403
2.274057523397343
1.7944915229429819
1.8816190684014829
2.436218265944448
0.8048420267382728
1.9027858959010246
1.8210256072381867
1.3661936117976894
2.629232386441093
2.277950845556791
2.545648571369276
1.4875318752002988
2.0449236673112132
2.845199607455242
1.886383002202385
2.378364997575332
2.0552819940974203
1.0636351880231556
1.1876555402464952
2.327143777166332
0.8191400595926267
1.7438930072335967
1.7567122762064977
0.5216826522663478
2.6240288653522756
1.980667350431514
1.4753652244242441
2.290529904992984
1.5570705963076166
1.8830896662619914
1.3807745354527383
2.619489737418417
1.7393879893097608
1.952002825638206
1.6608155571372178
1.718482499459071
1.9140658433659379
2.067059903144744
1.9910964568733664
1.3042274509227576
0.9976417639767793
1.3867308059406747
2.430169951113633
1.357989660252863
2.696778677058447
1.5563572021101966
2.214615134556377
2.0283619856418063
0.97103646788

1.0935832272652102
2.113876414125917
1.6398369233052492
2.766730129818932
2.3443120643351514
2.3443308594075756
1.2978633268640305
2.06926180540165
1.815870060230009
0.7458950031347943
2.2667422108555524
2.160856671697705
1.3849609410454227
1.434179293137782
1.4289392046904879
2.0021797461988404
1.8809529712097988
0.3597241511185995
0.6070314104343701
1.725748612257773
2.1333524992177364
1.4429234779265199
1.7434927823517286
2.588649368248295
2.332054572769734
2.5190225484461832
1.894302753732948
2.1598644748393627
2.474188753054779
1.6165480915183492
1.88999445864034
2.4642837788890284
2.0674474449354574
1.4394882158211992
2.091284714050834
1.7581233697677374
0.8613926833312128
2.170298324762003
1.7434351251840168
2.3629227618066047
2.6350722914614275
2.6758520296737154
0.5089389391414492
1.926398142307761
1.7971721783392443
2.239940009338603
2.565533818928515
2.0208952300933603
1.7652897972413255
2.254962151483503
1.4818766808930983
0.9078973171621137
2.0046945942904197
1.91530875789

2.4439746713066506
1.0949077888866008
2.1787980477865223
2.0907562617448057
1.7386486396784042
2.3926925365358276
2.4054916415467855
1.2591353731941832
1.4025655683194755
0.7642619273417632
2.8073680827977587
2.4072447467142686
2.1641851682809365
1.920893447903234
1.1089499545016583
1.8559046255594058
2.5233295982707933
1.8757621577916135
2.8167647804679814
2.6936577597577305
1.650038233343221
1.5537442897221392
1.7025539565986363
1.4720620104906428
2.3485716401474694
2.1294251839964202
1.6844845102509514
1.2985505957606043
1.6249702366887286
1.8253712723404898
2.1334970552659276
2.2570846896366508
0.4746224344539492
1.1817138431111305
2.205460401143409
2.236130495424704
2.125690166299817
2.1834415511050813
2.0696313555206043
1.9589997767237397
2.148093174223675
2.56556402952883
1.5813316214346236
2.4432401112530546
2.335679960983429
2.1115870106205654
1.0034012172319697
0.41281215075863364


In [29]:
FROGS = np.array(FROGS)
#FROGS = FROGS.reshape(1000,-1)

In [30]:
values = np.array(values)
values.shape

(5000, 9)

In [31]:
np.save("FROG_9p_5k", FROGS)

In [32]:
np.save("labels_ann_9p_5k", values)